In [4]:
import kwant
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import scipy.signal
from scipy.optimize import curve_fit
import scipy.signal
from scipy.stats import multivariate_normal
import argparse
import kwant.continuum


ham = (
    "hbar * vF * k_y * kron(sigma_x, sigma_z) - "
    "hbar * vF * k_x * kron(sigma_y, sigma_z) + "
    "{m} * kron(sigma_0, sigma_x) + " 
    "Msysz * kron(sigma_z, sigma_0)")


m = "(m0 - m1 * (k_x**2 + k_y**2))"
# hamiltonian_MTI=get_hamiltonian(material='mti', ph_symmetry=True)
hamiltonian_MTI = ham.format(m=m, Delta="Delta(x)")
ham_MTI_sys_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)
ham_MTI_leads_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)

In [3]:
def get_shape(L, W, type_sys='normal', v=np.array([0,0])):
    L_start, W_start = 0, 0
    L_stop, W_stop = L, W
    

    def _shape(site):
        (x, y) = site.pos - v
        return (0 <= x <= L and 0 <= y <= W)

    return _shape, v + np.array([L_stop, W_stop])
    
# def create_syst(L, W, type_sys='normal', v=np.array([0,0])):
    
a=10
W=100
L=500
syst = kwant.Builder()
L_start, W_start = 0, 0
L_stop, W_stop = L, W

# if type_sys=='normal':
def shape(L, W, v=np.array([0,0])):
    L_start, W_start = 0, 0
    L_stop, W_stop = L, W
    
    def _shape(site):
        (x, y) = site.pos - v
        return (0 <= x <= L and 0 <= y <= W)
    
    return _shape, v + np.array([L_stop, W_stop])

#     else:
#         def shape_pointcontact(W):
#             def _shape(site):
#                 (x, y) = site.pos
#                 return y==W//2

#             return _shape, (0,W//2)

m = "(m0 - m1 * (k_x**2 + k_y**2))"
# hamiltonian_MTI=get_hamiltonian(material='mti', ph_symmetry=True)
hamiltonian_MTI = ham.format(m=m, Delta="Delta(x)")
ham_MTI_sys_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)
ham_MTI_leads_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)

syst.fill(ham_MTI_sys_discretized, *_shape(L,W))
lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
# lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
lead_right = kwant.Builder(kwant.TranslationalSymmetry((a, 0)))  
lead_left.fill(ham_MTI_leads_discretized, *_shape(L, W))
lead_right.fill(ham_MTI_leads_discretized, *_shape(L, W))

syst.attach_lead(lead_left)
syst.attach_lead(lead_right)
fyst=syst.finalized()
    
    

NameError: name '_shape' is not defined

In [ ]:
*_shape(L,W)

In [ ]:
(10, 10) = site.pos - v    

In [ ]:
x=[1,2,3,4,5]

In [ ]:
*(x)

In [10]:
W=1000
L=11000

a = 10
syst = kwant.Builder()

hamiltonian_MTI = (
    "hbar * vF * k_y * kron(sigma_x, sigma_z) - "
    "hbar * vF * k_x * kron(sigma_y, sigma_z) + "
    "{m} * kron(sigma_0, sigma_x) + " # top-bottom surface coupling
    "Msysz * kron(sigma_z, sigma_0)"
)
                       

term_mimp = (
    " + S_mimp_x(site, Smag_mimp, Stheta_mimp, Sphi_mimp) * kron(sigma_x, sigma_0)"
    " + S_mimp_y(site, Smag_mimp, Stheta_mimp, Sphi_mimp) * kron(sigma_y, sigma_0)"
    " + S_mimp_z(site, Smag_mimp, Stheta_mimp, Sphi_mimp) * kron(sigma_z, sigma_0)"   
)


m = "(m0 - m1 * (k_x**2 + k_y**2))"

hamiltonian_MTI = hamiltonian_MTI.format(m=m)

ham_MTI_sys_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)
ham_MTI_leads_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)


def shape(L, W, v=np.array([0,0])):
    L_start, W_start = 0, 0
    L_stop, W_stop = L, W
    
    def _shape(site):
        (x, y) = site.pos - v
        return (0 <= x <= L and 0 <= y <= W)
    
    return _shape, v + np.array([L_stop, W_stop])
    
def shape_pointcontact(W):
    def _shape(site):
        (x, y) = site.pos
        return y==W//2
    
    return _shape, (0,W//2)

In [11]:
syst.fill(ham_MTI_sys_discretized, *shape(L,W))

lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
# lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
lead_right = kwant.Builder(kwant.TranslationalSymmetry((a, 0)))  
lead_left.fill(ham_MTI_leads_discretized, *shape(L, W))
lead_right.fill(ham_MTI_leads_discretized, *shape(L, W))

print(syst)
syst.attach_lead(lead_left)
syst.attach_lead(lead_right)

fsyst=syst.finalized()

# Bfields=np.linspace
# for B in Bfields:

params = dict(hbar=1,
                    V=0,
                    mu_m=1.6,
                    vF=3,#for Bi2Se3-based 3DTI, 10^15 canceled out with hbar
                    m0=-0.005,
                    m1=15,
                    Msysz=0.015,
                    Msysx=.0,
                    Msysy=.0,
                    Mz=0.1,
                    Mx=.0,
                    My=.0,
                    a=a,
                    phi_0=1.0,
                    exp=np.exp,
                    re=lambda x: x.real,
                    im=lambda x: x.imag,
                    Delta=lambda x: 0.005 if x >= 1000 else 0
                    )


# smatrix = kwant.smatrix(fsyst, energy=3.7923819182006433, params = params)
# E_data.append(smatrix.transmission((1, 0), (0,0)))
# np.savetxt(f"NEW_QAHvsW_L1100nm_cor50{N}.csv", E_data, delimiter=',')

In [14]:
smatrix = kwant.smatrix(fsyst, energy=0.05, params = params)

In [16]:
smatrix.transmission((1, 0), (0,0))

10.000000000005459

In [17]:
fsyst

In [18]:
def syst_create(L, W):
    syst.fill(ham_MTI_sys_discretized, *shape(L,W))

    lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
    # lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
    lead_right = kwant.Builder(kwant.TranslationalSymmetry((a, 0)))  
    lead_left.fill(ham_MTI_leads_discretized, *shape(L, W))
    lead_right.fill(ham_MTI_leads_discretized, *shape(L, W))

    print(syst)
    syst.attach_lead(lead_left)
    syst.attach_lead(lead_right)
    fsyst=syst.finalized()
    
    return fsyst

In [26]:
fsyst=syst_create(1100,400)
fsyst

/var/tmp/pbs.8892112.pbs/ipykernel_3125591/1698737364.py:2: RuntimeWarning: fill(): The target builder already contains all starting sites.
  syst.fill(ham_MTI_sys_discretized, *shape(L,W))


In [24]:
def create_syst(L, W, type_sys='normal', v=np.array([0,0])):
    
    a=10
    syst = kwant.Builder()
    L_start, W_start = 0, 0
    L_stop, W_stop = L, W


        def _shape(site):
            (x, y) = site.pos - v
            return (0 <= x <= L and 0 <= y <= W)

        return _shape, v + np.array([L_stop, W_stop])
    
#     else:
#         def shape_pointcontact(W):
#             def _shape(site):
#                 (x, y) = site.pos
#                 return y==W//2

#             return _shape, (0,W//2)
        
        m = "(m0 - m1 * (k_x**2 + k_y**2))"
        hamiltonian_MTI=get_hamiltonian(material='mti', ph_symmetry=True)
        hamiltonian_MTI = hamiltonian_MTI.format(m=m, Delta="Delta(x)")
        ham_MTI_sys_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)
        ham_MTI_leads_discretized = kwant.continuum.discretize(hamiltonian_MTI, grid=a)

        syst.fill(ham_MTI_sys_discretized, *_shape(L,W))
        lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
        # lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
        lead_right = kwant.Builder(kwant.TranslationalSymmetry((a, 0)))  
        lead_left.fill(ham_MTI_leads_discretized, *_shape(L, W))
        lead_right.fill(ham_MTI_leads_discretized, *_shape(L, W))

        syst.attach_lead(lead_left)
        syst.attach_lead(lead_right)
        fyst=syst.finalized()
    
    
        return fsyst

IndentationError: unexpected indent (1338030485.py, line 9)

In [22]:
create_syst(L, W, type_sys='normal', v=np.array([0,0]))

(<function __main__.create_syst.<locals>._shape(site)>, array([11000,  1000]))

In [5]:
a=10
W=100
L=500
syst = kwant.Builder()
L_start, W_start = 0, 0
L_stop, W_stop = L, W


def shape(L, W, v=np.array([0,0])):
    L_start, W_start = 0, 0
    L_stop, W_stop = L, W
    
    def _shape(site):
        (x, y) = site.pos - v
        return (0 <= x <= L and 0 <= y <= W)
    
    return _shape, v + np.array([L_stop, W_stop])

def syst_create(L, W):
    a=10
    W=W
    L=L
    syst = kwant.Builder()
    L_start, W_start = 0, 0
    L_stop, W_stop = L, W
    syst.fill(ham_MTI_sys_discretized, *shape(L,W))

    lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
    # lead_left = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
    lead_right = kwant.Builder(kwant.TranslationalSymmetry((a, 0)))  
    lead_left.fill(ham_MTI_leads_discretized, *shape(L, W))
    lead_right.fill(ham_MTI_leads_discretized, *shape(L, W))

    print(syst)
    syst.attach_lead(lead_left)
    syst.attach_lead(lead_right)
    fsyst=syst.finalized()
    
    return fsyst

In [7]:
fsyst=syst_create(1100, 400)

/var/tmp/pbs.8892112.pbs/ipykernel_3135107/743879433.py:20: RuntimeWarning: fill(): The target builder already contains all starting sites.
  syst.fill(ham_MTI_sys_discretized, *shape(L,W))


In [8]:
fsyst

In [10]:
params = dict(hbar=1,
                    V=0,
                    mu_m=1.6,
                    vF=3,#for Bi2Se3-based 3DTI, 10^15 canceled out with hbar
                    m0=-0.005,
                    m1=15,
                    Msysz=0.015,
                    Msysx=.0,
                    Msysy=.0,
                    Mz=0.1,
                    Mx=.0,
                    My=.0,
                    a=a,
                    phi_0=1.0,
                    exp=np.exp,
                    re=lambda x: x.real,
                    im=lambda x: x.imag,
                    Delta=lambda x: 0.005 if x >= 1000 else 0
                    )


smatrix = kwant.smatrix(fsyst, energy=0.05, params = params)
smatrix.transmission((1, 0), (0,0))

0.7074644594106853